In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/Original/projectImages-20250614T025212Z-1-001.zip" -d "/content/drive/MyDrive/Filtered Fashion images"

In [ ]:
import os
generated_folder="/content/drive/MyDrive/Images/Generated"
original_folder="/content/drive/MyDrive/Images/Original"
output_base = '/content/drive/MyDrive/Filtered Fashion images/projectImages/dataset_split'
train_generated = os.path.join(output_base, 'train/generated')
train_original = os.path.join(output_base, 'train/original')
test_generated = os.path.join(output_base, 'test/generated')
test_original = os.path.join(output_base, 'test/original')

# Create output directories
for path in [train_generated, train_original, test_generated, test_original]:
    os.makedirs(path, exist_ok=True)

In [ ]:
# # List all files
# import shutil
# from sklearn.model_selection import train_test_split
# generated_files = [f for f in os.listdir(generated_folder) if os.path.isfile(os.path.join(generated_folder, f))]
# original_files = [f for f in os.listdir(original_folder) if os.path.isfile(os.path.join(original_folder, f))]

# train_files, test_files = train_test_split(generated_files, test_size=0.1, random_state=42)
# trainoriginal_files, testoriginal_files = train_test_split(original_files, test_size=0.1, random_state=42)
# def copy_files(file_list, src_folder, dest_folder):
#     for file in file_list:
#         src_path = os.path.join(src_folder, file)
#         dest_path = os.path.join(dest_folder, file)
#         if os.path.exists(src_path):
#             shutil.copy(src_path, dest_path)
#         else:
#             print(f"Warning: {file} not found in {src_folder}")


# # Corrected copy calls
# copy_files(train_files, generated_folder, train_generated)
# copy_files(test_files, generated_folder, test_generated)

# copy_files(trainoriginal_files, original_folder, train_original)  # fixed
# copy_files(testoriginal_files, original_folder, test_original)    # fixed

# print(f"Done! {len(train_files)} for training, {len(test_files)} for testing.")

Done! 864 for training, 97 for testing.


In [ ]:
import os

# Define the folders
train_original_folder = "/content/drive/MyDrive/Filtered Fashion images/projectImages/dataset_split/train/original"
train_generated_folder = "/content/drive/MyDrive/Filtered Fashion images/projectImages/dataset_split/train/generated"
test_original_folder = "/content/drive/MyDrive/Filtered Fashion images/projectImages/dataset_split/test/original"
test_generated_folder = "/content/drive/MyDrive/Filtered Fashion images/projectImages/dataset_split/test/generated"

# Define valid image extensions
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')

# Count images in a folder
def count_images(folder):
    return sum(1 for file in os.listdir(folder) if file.lower().endswith(image_extensions))

# Get the counts
train_original_count = count_images(train_original_folder)
train_generated_count = count_images(train_generated_folder)
test_original_count = count_images(test_original_folder)
test_generated_count = count_images(test_generated_folder)

# Print the counts
print(f"Train - Original: {train_original_count} | Generated: {train_generated_count}")
print(f"Test  - Original: {test_original_count} | Generated: {test_generated_count}")


Train - Original: 7437 | Generated: 7460
Test  - Original: 827 | Generated: 829


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models, optimizers, callbacks

train_dir = os.path.join(output_base, 'train')
test_dir  = os.path.join(output_base, 'test')
IMG_SIZE = (224, 224)       # ResNet50 default input size is 224x224
BATCH_SIZE = 32
SEED = 42


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.1,      # reserve 10% of train for validation
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=SEED
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False,
    seed=SEED
)


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)


Found 13408 images belonging to 2 classes.
Found 1489 images belonging to 2 classes.
Found 1656 images belonging to 2 classes.
Class indices: {'generated': 0, 'original': 1}


In [ ]:
#build resnet50 model
base_model = ResNet50(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False
# Add classification head
inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)  # ensure batchnorm runs in inference mode
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()
# original
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=1e-4),
#     loss='binary_crossentropy',
#     metrics=['accuracy',
#              tf.keras.metrics.Precision(name='precision'),
#              tf.keras.metrics.Recall(name='recall'),
#              tf.keras.metrics.AUC(name='auc'),
#              F1Score(name='f1_score')],

#     run_eagerly=True

# )

# test
model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=[
      'accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall   (name='recall'),
      tf.keras.metrics.AUC      (name='auc'),
      F1Score(name='f1_score')
    ]
)

In [ ]:
checkpoint_path = '/content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5'

os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
checkpoint_cb = callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_auc',
    mode='max',
    save_best_only=True,
    verbose=1
)
earlystop_cb = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1
)


In [ ]:
# original
# history = model.fit(
#     train_generator,
#     epochs=20,
#     steps_per_epoch=len(train_generator),
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator),
#     callbacks=[checkpoint_cb, earlystop_cb]
# )

# # And after training:
# # Evaluate
# results = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
# print("Metrics:", dict(zip(model.metrics_names, results)))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 953ms/step - accuracy: 0.5603 - auc: 0.5912 - f1_score: 0.5211 - loss: 0.7894 - precision: 0.5621 - recall: 0.4966
Epoch 1: val_auc improved from -inf to 0.98486, saving model to /content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5


49/49 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5618 - auc: 0.5931 - f1_score: 0.5232 - loss: 0.7871 - precision: 0.5637 - recall: 0.4989 - val_accuracy: 0.8547 - val_auc: 0.9849 - val_f1_score: 0.8731 - val_loss: 0.3893 - val_precision: 0.7748 - val_recall: 1.0000
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 991ms/step - accuracy: 0.8153 - auc: 0.8958 - f1_score: 0.8217 - loss: 0.4145 - precision: 0.8036 - recall: 0.8413
Epoch 2: val_auc improved from 0.98486 to 0.99655, saving model to /content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5


49/49 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.8154 - auc: 0.8960 - f1_score: 0.8217 - loss: 0.4140 - precision: 0.8037 - recall: 0.8413 - val_accuracy: 0.9244 - val_auc: 0.9966 - val_f1_score: 0.9297 - val_loss: 0.2799 - val_precision: 0.8687 - val_recall: 1.0000
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8932 - auc: 0.9530 - f1_score: 0.8951 - loss: 0.2993 - precision: 0.8850 - recall: 0.9055
Epoch 3: val_auc did not improve from 0.99655
49/49 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.8933 - auc: 0.9531 - f1_score: 0.8952 - loss: 0.2990 - precision: 0.8852 - recall: 0.9055 - val_accuracy: 0.9012 - val_auc: 0.9947 - val_f1_score: 0.9101 - val_loss: 0.2468 - val_precision: 0.8350 - val_recall: 1.0000
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 990ms/step - accuracy: 0.9063 - auc: 0.9730 - f1_score: 0.9098 - loss: 0.2372 - precision: 0.8922 - recall: 0.9286
Epoch 4: val_auc improved from 0.99655 to 0.99905, saving model to /content/drive/MyDrive/projectImages

49/49 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9065 - auc: 0.9731 - f1_score: 0.9100 - loss: 0.2370 - precision: 0.8925 - recall: 0.9287 - val_accuracy: 0.9651 - val_auc: 0.9991 - val_f1_score: 0.9659 - val_loss: 0.1725 - val_precision: 0.9444 - val_recall: 0.9884
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9373 - auc: 0.9890 - f1_score: 0.9363 - loss: 0.1703 - precision: 0.9413 - recall: 0.9317
Epoch 5: val_auc improved from 0.99905 to 0.99966, saving model to /content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5


49/49 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.9375 - auc: 0.9890 - f1_score: 0.9366 - loss: 0.1702 - precision: 0.9413 - recall: 0.9322 - val_accuracy: 0.9767 - val_auc: 0.9997 - val_f1_score: 0.9773 - val_loss: 0.1416 - val_precision: 0.9556 - val_recall: 1.0000
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9568 - auc: 0.9907 - f1_score: 0.9553 - loss: 0.1513 - precision: 0.9551 - recall: 0.9563
Epoch 6: val_auc did not improve from 0.99966
49/49 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.9567 - auc: 0.9907 - f1_score: 0.9553 - loss: 0.1513 - precision: 0.9549 - recall: 0.9565 - val_accuracy: 0.9826 - val_auc: 0.9994 - val_f1_score: 0.9829 - val_loss: 0.1201 - val_precision: 0.9663 - val_recall: 1.0000
Epoch 7/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 959ms/step - accuracy: 0.9641 - auc: 0.9946 - f1_score: 0.9643 - loss: 0.1241 - precision: 0.9505 - recall: 0.9786
Epoch 7: val_auc improved from 0.99966 to 1.00000, saving model to /content/drive/MyDrive/projectImages

49/49 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9640 - auc: 0.9945 - f1_score: 0.9642 - loss: 0.1243 - precision: 0.9505 - recall: 0.9785 - val_accuracy: 0.9826 - val_auc: 1.0000 - val_f1_score: 0.9829 - val_loss: 0.0993 - val_precision: 0.9663 - val_recall: 1.0000
Epoch 8/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 955ms/step - accuracy: 0.9692 - auc: 0.9949 - f1_score: 0.9687 - loss: 0.1075 - precision: 0.9782 - recall: 0.9595
Epoch 8: val_auc did not improve from 1.00000
49/49 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9692 - auc: 0.9949 - f1_score: 0.9687 - loss: 0.1074 - precision: 0.9780 - recall: 0.9597 - val_accuracy: 0.9535 - val_auc: 0.9974 - val_f1_score: 0.9551 - val_loss: 0.1213 - val_precision: 0.9239 - val_recall: 0.9884
Epoch 9/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 968ms/step - accuracy: 0.9615 - auc: 0.9931 - f1_score: 0.9619 - loss: 0.1119 - precision: 0.9643 - recall: 0.9600
Epoch 9: val_auc did not improve from 1.00000
49/49 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.9

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 742ms/step - accuracy: 0.7721 - auc: 0.8481 - f1_score: 0.7743 - loss: 0.4573 - precision: 0.7666 - recall: 0.7822
Epoch 1: val_auc improved from -inf to 0.99825, saving model to /content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5


419/419 ━━━━━━━━━━━━━━━━━━━━ 366s 831ms/step - accuracy: 0.7723 - auc: 0.8483 - f1_score: 0.7745 - loss: 0.4570 - precision: 0.7668 - recall: 0.7825 - val_accuracy: 0.9772 - val_auc: 0.9983 - val_f1_score: 0.9772 - val_loss: 0.1133 - val_precision: 0.9758 - val_recall: 0.9785 - learning_rate: 1.0000e-04
Epoch 2/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 584ms/step - accuracy: 0.9468 - auc: 0.9885 - f1_score: 0.9472 - loss: 0.1443 - precision: 0.9438 - recall: 0.9505
Epoch 2: val_auc improved from 0.99825 to 0.99862, saving model to /content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5


419/419 ━━━━━━━━━━━━━━━━━━━━ 273s 651ms/step - accuracy: 0.9469 - auc: 0.9885 - f1_score: 0.9472 - loss: 0.1442 - precision: 0.9439 - recall: 0.9506 - val_accuracy: 0.9839 - val_auc: 0.9986 - val_f1_score: 0.9838 - val_loss: 0.0727 - val_precision: 0.9838 - val_recall: 0.9838 - learning_rate: 1.0000e-04
Epoch 3/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - accuracy: 0.9708 - auc: 0.9960 - f1_score: 0.9704 - loss: 0.0875 - precision: 0.9669 - recall: 0.9740
Epoch 3: val_auc improved from 0.99862 to 0.99957, saving model to /content/drive/MyDrive/projectImages/resnet50_finetune/checkpoint.h5


419/419 ━━━━━━━━━━━━━━━━━━━━ 267s 637ms/step - accuracy: 0.9708 - auc: 0.9960 - f1_score: 0.9704 - loss: 0.0875 - precision: 0.9669 - recall: 0.9740 - val_accuracy: 0.9879 - val_auc: 0.9996 - val_f1_score: 0.9878 - val_loss: 0.0517 - val_precision: 0.9945 - val_recall: 0.9812 - learning_rate: 1.0000e-04
Epoch 4/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.9719 - auc: 0.9965 - f1_score: 0.9721 - loss: 0.0768 - precision: 0.9710 - recall: 0.9732
Epoch 4: val_auc did not improve from 0.99957
419/419 ━━━━━━━━━━━━━━━━━━━━ 264s 630ms/step - accuracy: 0.9719 - auc: 0.9965 - f1_score: 0.9721 - loss: 0.0768 - precision: 0.9710 - recall: 0.9732 - val_accuracy: 0.9819 - val_auc: 0.9993 - val_f1_score: 0.9815 - val_loss: 0.0571 - val_precision: 0.9972 - val_recall: 0.9664 - learning_rate: 1.0000e-04
Epoch 5/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 559ms/step - accuracy: 0.9767 - auc: 0.9973 - f1_score: 0.9769 - loss: 0.0667 - precision: 0.9740 - recall: 0.9798
Epoch 5: val_auc improved fr

419/419 ━━━━━━━━━━━━━━━━━━━━ 322s 629ms/step - accuracy: 0.9767 - auc: 0.9973 - f1_score: 0.9769 - loss: 0.0666 - precision: 0.9740 - recall: 0.9798 - val_accuracy: 0.9933 - val_auc: 0.9999 - val_f1_score: 0.9933 - val_loss: 0.0296 - val_precision: 0.9959 - val_recall: 0.9906 - learning_rate: 1.0000e-04
Epoch 6/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 557ms/step - accuracy: 0.9788 - auc: 0.9966 - f1_score: 0.9790 - loss: 0.0654 - precision: 0.9762 - recall: 0.9819
Epoch 6: val_auc did not improve from 0.99990
419/419 ━━━━━━━━━━━━━━━━━━━━ 275s 655ms/step - accuracy: 0.9788 - auc: 0.9966 - f1_score: 0.9790 - loss: 0.0654 - precision: 0.9762 - recall: 0.9819 - val_accuracy: 0.9872 - val_auc: 0.9992 - val_f1_score: 0.9872 - val_loss: 0.0446 - val_precision: 0.9918 - val_recall: 0.9825 - learning_rate: 1.0000e-04
Epoch 7/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 559ms/step - accuracy: 0.9794 - auc: 0.9982 - f1_score: 0.9793 - loss: 0.0550 - precision: 0.9787 - recall: 0.9799
Epoch 7: val_auc did not imp

419/419 ━━━━━━━━━━━━━━━━━━━━ 263s 626ms/step - accuracy: 0.9866 - auc: 0.9990 - f1_score: 0.9865 - loss: 0.0366 - precision: 0.9854 - recall: 0.9875 - val_accuracy: 0.9946 - val_auc: 0.9999 - val_f1_score: 0.9946 - val_loss: 0.0238 - val_precision: 1.0000 - val_recall: 0.9892 - learning_rate: 1.0000e-04
Epoch 16/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 555ms/step - accuracy: 0.9891 - auc: 0.9994 - f1_score: 0.9891 - loss: 0.0298 - precision: 0.9901 - recall: 0.9881
Epoch 16: val_auc did not improve from 0.99993

Epoch 16: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
419/419 ━━━━━━━━━━━━━━━━━━━━ 258s 616ms/step - accuracy: 0.9891 - auc: 0.9994 - f1_score: 0.9891 - loss: 0.0298 - precision: 0.9901 - recall: 0.9881 - val_accuracy: 0.9899 - val_auc: 0.9998 - val_f1_score: 0.9899 - val_loss: 0.0237 - val_precision: 0.9932 - val_recall: 0.9865 - learning_rate: 1.0000e-04
Epoch 17/20
419/419 ━━━━━━━━━━━━━━━━━━━━ 0s 552ms/step - accuracy: 0.9860 - auc: 0.9992 - f1_score: 0.9857

In [ ]:
# 4) Single evaluate call:
results = model.evaluate(
    test_generator,
    steps=len(test_generator),
    verbose=1,
    return_dict=True
)
print(results)

52/52 ━━━━━━━━━━━━━━━━━━━━ 16s 313ms/step - accuracy: 1.0000 - auc: 0.5283 - f1_score: 0.5283 - loss: 0.0016 - precision: 0.5283 - recall: 0.5283
{'accuracy': 1.0, 'auc': 1.0000001192092896, 'f1_score': 1.0, 'loss': 0.0015415140660479665, 'precision': 1.0, 'recall': 1.0}


In [ ]:
# results = model.evaluate(
#     test_generator,
#     steps=len(test_generator),
#     verbose=1
# )

# # See in what order they come back:
# print(model.metrics_names)
# # e.g. ['loss', 'accuracy', 'precision', 'recall', 'auc', 'f1_score']

# loss, accuracy, precision, recall, auc, f1_score = results

# print(f"Test → loss: {loss:.4f}")
# print(f"       accuracy: {accuracy:.3f}")
# print(f"       precision: {precision:.3f}")
# print(f"       recall: {recall:.3f}")
# print(f"       AUC: {auc:.3f}")
# print(f"       F1‑score: {f1_score:.3f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 597ms/step - accuracy: 1.0000 - auc: 0.6250 - f1_score: 0.6250 - loss: 0.0071 - precision: 0.6250 - recall: 0.6250
['loss', 'compile_metrics']
Test → loss: 0.0078
       accuracy: 1.000
       precision: 1.000
       recall: 1.000
       AUC: 1.000
       F1‑score: 1.000
